In [4]:
import pickle
import pandas as pd

In [43]:
df = pd.read_csv("data/test.csv")

In [44]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26
2,2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70
3,3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59
4,4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67


In [45]:
# estandarización
with open('data/estandarizacion.pkl', 'rb') as estandarizacion:
    estandarizacion = pickle.load(estandarizacion)

In [46]:
with open('data/tree_sinprecios.pkl', 'rb') as modelo:
    modelo = pickle.load(modelo)

In [47]:
df.sample(4)

,id,carat,cut,color,clarity,depth,table,x,y,z
880,880,0.30,Very Good,H,SI1,63.4,58.0,4.24,4.21,2.68
1893,1893,1.04,Ideal,G,VS1,61.5,55.0,6.54,6.55,4.02
983,983,0.53,Ideal,H,VS2,61.4,56.0,5.20,5.23,3.20
9486,9486,1.03,Very Good,G,VS1,61.3,56.1,6.50,6.52,3.99


In [48]:
orden_cut = ['Ideal', 'Good', 'Very Good','Premium' , 'Fair' ]
orden_color = ['E','D',  'F', 'G', 'H', 'I', 'J']
orden_clarity = ['VVS1','IF','VS1','SI1','SI2','VS2','I1', 'VVS2'] 

In [49]:

from sklearn.preprocessing import OrdinalEncoder

In [50]:
def ordinal_encoder(orden, df, columna):
    ordinal = OrdinalEncoder(categories = [orden], dtype = int)
    transformados_oe = ordinal.fit_transform(df[[columna]])
    df[columna] = transformados_oe
    return df

In [51]:
df = ordinal_encoder(orden_cut, df, "cut")
df = ordinal_encoder(orden_color, df, "color")
df = ordinal_encoder(orden_clarity, df, "clarity")

In [52]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,0,5,3,60.5,58.0,4.43,4.49,2.70
1,1,1.24,3,5,3,62.9,60.0,6.80,6.74,4.26
2,2,1.66,3,1,3,62.0,59.0,7.55,7.60,4.70
3,3,0.75,3,1,4,60.6,56.0,5.94,5.90,3.59
4,4,1.50,4,0,4,64.8,55.0,7.26,7.15,4.67


In [53]:
estandarizadas = pd.DataFrame(estandarizacion.transform(df[['carat', 'depth', 'table', 'x', 'y',
       'z']]), columns = ['carat', 'depth', 'table', 'x', 'y',
       'z'])

In [55]:
estandarizadas.columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [56]:
df[['carat', 'depth', 'table', 'x', 'y', 'z']] = estandarizadas[['carat', 'depth', 'table', 'x', 'y', 'z']]

In [57]:
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,-1.003451,0,5,3,-0.867480,0.245377,-1.154576,-1.076301,-1.177719
1,1,0.929593,3,5,3,0.802923,1.140416,0.951660,0.872114,1.016394
2,2,1.812070,3,1,3,0.176522,0.692896,1.618191,1.616841,1.635246
3,3,-0.099963,3,1,4,-0.797880,-0.649661,0.187372,0.144706,0.074050
4,4,1.475888,4,0,4,2.125326,-1.097180,1.360465,1.227158,1.593051


In [63]:
df_final = df.drop("id",axis=1)

In [64]:
df_final.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,-1.003451,0,5,3,-0.867480,0.245377,-1.154576,-1.076301,-1.177719
1,0.929593,3,5,3,0.802923,1.140416,0.951660,0.872114,1.016394
2,1.812070,3,1,3,0.176522,0.692896,1.618191,1.616841,1.635246
3,-0.099963,3,1,4,-0.797880,-0.649661,0.187372,0.144706,0.074050
4,1.475888,4,0,4,2.125326,-1.097180,1.360465,1.227158,1.593051


In [60]:
from sklearn.preprocessing import StandardScaler
import numpy as np

In [82]:
precio_final=modelo.predict(df_final)

In [83]:
precio_final

array([6.03733333, 8.58510526, 9.48688889, ..., 6.40011111, 8.7028    ,
       8.02266667])

In [84]:
precio_final = pd.DataFrame({'price': precio_final})

In [85]:
precio_final.head()

,price
0,6.037333
1,8.585105
2,9.486889
3,7.813636
4,8.931300


In [86]:
df_precios_finales = pd.concat([df["id"], precio_final], axis=1)

In [87]:
df_precios_finales.tail()

,id,price
13480,13480,8.551111
13481,13481,8.441667
13482,13482,6.400111
13483,13483,8.702800
13484,13484,8.022667


In [88]:
df_precios_finales.to_csv("data/Submission_sinstandarizar.csv", index=False)